# Python Apache Spark

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f
from pyspark.sql.window import Window 

In [2]:
spark = SparkSession.builder \
    .appName("RetailSalesAnalysis") \
    .getOrCreate()

In [3]:
df = spark.read.csv("dados/global_retail_sales_data.csv", header=True, inferSchema=True)

In [4]:
df.printSchema()

root
 |-- Transaction ID: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Store ID: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Quantity Sold: integer (nullable = true)
 |-- Price: double (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Total Sales: double (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Payment Method: string (nullable = true)
 |-- Store Location: string (nullable = true)
 |-- Customer Age: integer (nullable = true)
 |-- Customer Gender: string (nullable = true)
 |-- Customer Segment: string (nullable = true)
 |-- Loyalty Program: string (nullable = true)
 |-- Feedback Rating: integer (nullable = true)
 |-- Employee ID: string (nullable = true)
 |-- Store Type: string (nullable = true)



In [5]:
df.limit(5).toPandas()

,Transaction ID,Date,Store ID,Product ID,Category,Sub-Category,Quantity Sold,Price,Discount,Total Sales,Customer ID,Payment Method,Store Location,Customer Age,Customer Gender,Customer Segment,Loyalty Program,Feedback Rating,Employee ID,Store Type
0,T0000001,2023-10-13,S074,P00991,Groceries,Dairy,7,56.49,0.20,316.34,d49b8699-04cd-424c-8ffa-dd6210950328,Online Payment,"Martinstad, Guinea",41,Female,VIP,No,2,E0258,Online
1,T0000002,2022-12-13,S044,P00266,Groceries,Snacks,1,521.88,0.03,506.22,e46ee47a-c8d8-488d-89a2-3c7f58090a46,Online Payment,"New Micheleview, Paraguay",68,Female,Regular,No,3,E0081,Online
2,T0000003,2022-09-15,S030,P00711,Groceries,Fruits,6,961.08,0.02,5651.15,01255bf5-ae89-45bc-99dc-6aade26b41f9,Cash,"Martinstad, Guinea",47,Male,VIP,Yes,2,E0446,Online
3,T0000004,2023-02-28,S042,P00733,Furniture,Tables,8,417.71,0.26,2472.84,a62979a4-9370-4b08-abac-8ed0284115ab,Cash,"Lake Diana, Nepal",37,Female,VIP,Yes,5,E0152,Outlet
4,T0000005,2023-12-12,S010,P00815,Electronics,Laptops,8,70.94,0.28,408.61,296c62a6-e926-4870-9bce-219275fa542f,Online Payment,"Port Melanie, Moldova",21,Male,Regular,No,4,E0055,Outlet


### Q1. How many transactions were recorded in the dataset?

In [6]:
df.count()

10000

### Q2. What is the total revenue generated across all transactions?

In [7]:
df.agg(f.sum("Total Sales").alias("Total Revenue")).show()

+--------------------+
|       Total Revenue|
+--------------------+
|2.3508131990000144E7|
+--------------------+



### Q3. How many unique products are there in the dataset?

In [8]:
df.select("Product ID").distinct().count()

1000

### Q4. Which product category has the most transactions?

In [9]:
df.groupBy("Category").count().orderBy(f.col("count").desc()).show(1)

+-----------+-----+
|   Category|count|
+-----------+-----+
|Electronics| 2035|
+-----------+-----+
only showing top 1 row



### Q5. How many different payment methods were used in the dataset?

In [10]:
df.select("Payment Method").distinct().count()

3

### Q6. What is the average transaction amount?

In [11]:
df.agg(f.avg("Total Sales").alias("Average Transaction Amount")).show()

+--------------------------+
|Average Transaction Amount|
+--------------------------+
|        2350.8131990000143|
+--------------------------+



### Q7. Which store has the highest total sales?

In [12]:
df.groupBy("Store ID").agg(f.sum("Total Sales").alias("Total Sales")).orderBy(f.col("Total Sales").desc()).show(1)

+--------+-----------------+
|Store ID|      Total Sales|
+--------+-----------------+
|    S039|295627.3300000001|
+--------+-----------------+
only showing top 1 row



### Q8. What is the average discount applied across all transactions?

In [13]:
df.agg(f.avg("Discount").alias("Avarage Discount")).show()

+-------------------+
|   Avarage Discount|
+-------------------+
|0.15025099999999697|
+-------------------+



### Q9. How many transactions were made by VIP customers?

In [14]:
df.filter(f.col("Customer Segment") == "VIP").count()

4895

### Q10. Which store type (e.g., Supermarket, Online, Outlet) has the highest average feedback rating?

In [15]:
df.groupBy("Store Type").agg(f.avg("Feedback Rating").alias("Avarage Feedback Rating")).orderBy(f.col("Avarage Feedback Rating").desc()).show(1)

+----------+-----------------------+
|Store Type|Avarage Feedback Rating|
+----------+-----------------------+
|    Outlet|     3.0015073861923427|
+----------+-----------------------+
only showing top 1 row



### Q11. What is the total sales amount for each store across different months?

In [16]:
# Extract the year and month from the Date column
df = df.withColumn("Year", f.year(f.col("Date")))
df = df.withColumn("Month", f.month(f.col("Date")))

In [17]:
# Group by Store ID, Year, and Month and calculate the total sales
total_sales_per_store_month = df.groupBy("Store ID", "Year", "Month") \
                                .agg(f.sum("Total Sales").alias("Total Sales"))

In [18]:
# Show the result
total_sales_per_store_month.orderBy("Store ID", "Year", "Month").show()

+--------+----+-----+------------------+
|Store ID|Year|Month|       Total Sales|
+--------+----+-----+------------------+
|    S001|2022|    9| 5989.860000000001|
|    S001|2022|   10|          17982.27|
|    S001|2022|   11|           4408.25|
|    S001|2022|   12|16871.569999999996|
|    S001|2023|    1|          10861.85|
|    S001|2023|    2|           11993.2|
|    S001|2023|    3|            5205.3|
|    S001|2023|    4|            8344.8|
|    S001|2023|    5|          16104.75|
|    S001|2023|    6|           4629.76|
|    S001|2023|    7|          17687.41|
|    S001|2023|    8|           8633.63|
|    S001|2023|    9|           10148.9|
|    S001|2023|   10|          19128.98|
|    S001|2023|   11|10259.849999999999|
|    S001|2023|   12|          16521.89|
|    S001|2024|    1|21536.999999999996|
|    S001|2024|    2|           4073.62|
|    S001|2024|    3|            4911.1|
|    S001|2024|    4|14524.789999999999|
+--------+----+-----+------------------+
only showing top

### Q12. Which store has the highest average feedback rating?

In [19]:
# Group by Store ID and calculate the average feedback rating
avg_feedback_per_store = df.groupBy("Store ID") \
                           .agg(f.avg("Feedback Rating").alias("Average Feedback Rating"))

# Order by Average Feedback Rating in descending order and get the top store
top_store = avg_feedback_per_store.orderBy(f.col("Average Feedback Rating").desc()).limit(1)

# Show the result
top_store.show()

+--------+-----------------------+
|Store ID|Average Feedback Rating|
+--------+-----------------------+
|    S005|     3.4234234234234235|
+--------+-----------------------+



### Q13. How does the average discount percentage vary by product category?

In [20]:
# Group by Category and calculate the average discount percentage
avg_discount_per_category = df.groupBy("Category") \
                              .agg(f.avg("Discount").alias("Average Discount"))

# Order the result by Average Discount for better readability (optional)
avg_discount_per_category = avg_discount_per_category.orderBy(f.col("Average Discount").desc())

# Show the result
avg_discount_per_category.show()

+-----------+-------------------+
|   Category|   Average Discount|
+-----------+-------------------+
|   Clothing|0.15178286852589584|
|Electronics|0.15120393120393072|
|       Toys|0.15084521384928679|
|  Furniture|0.14925607287449394|
|  Groceries|0.14816063460585016|
+-----------+-------------------+



### Q14. What is the most popular payment method used by customers in different countries?

In [21]:
# Extract the country from the store location (assuming store location format is "City, Country")
df = df.withColumn("Country", f.split(f.col("Store Location"), ", ").getItem(1))

# Group by Country and Payment Method, and count the transactions
payment_method_count = df.groupBy("Country", "Payment Method") \
                         .agg(f.count("Transaction ID").alias("Count"))

# Use a window function to determine the most popular payment method per country
window = Window.partitionBy("Country").orderBy(f.col("Count").desc())

# Add a rank to determine the most popular payment method
ranked_payment_methods = payment_method_count.withColumn("Rank", f.row_number().over(window))

# Filter to keep only the most popular payment method per country
most_popular_payment_method = ranked_payment_methods.filter(f.col("Rank") == 1)

# Show the result
most_popular_payment_method.select("Country", "Payment Method", "Count").show()

+--------------------+--------------+-----+
|             Country|Payment Method|Count|
+--------------------+--------------+-----+
|         Afghanistan|Online Payment|   39|
|             Albania|   Credit Card|   80|
|             Andorra|          Cash|   42|
|              Angola|Online Payment|   39|
|            Anguilla|   Credit Card|   35|
|Antarctica (the t...|          Cash|   35|
| Antigua and Barbuda|Online Payment|   35|
|             Austria|   Credit Card|   50|
|             Bahamas|   Credit Card|   38|
|             Bahrain|Online Payment|   43|
|             Belarus|   Credit Card|   34|
|               Benin|Online Payment|   44|
|             Bermuda|          Cash|   39|
|              Bhutan|Online Payment|   81|
|British Virgin Is...|   Credit Card|   39|
|                Chad|Online Payment|   38|
|               Chile|          Cash|   40|
|    Christmas Island|   Credit Card|   35|
|Cocos (Keeling) I...|Online Payment|   46|
|            Colombia|          

### Q15. Identify the top 10 products with the highest total sales.

In [22]:
# Group by Product ID and calculate the total sales
total_sales_per_product = df.groupBy("Product ID") \
                            .agg(f.sum("Total Sales").alias("Total Sales"))

# Order by Total Sales in descending order and get the top 10 products
top_10_products = total_sales_per_product.orderBy(f.col("Total Sales").desc()).limit(10)

# Show the result
top_10_products.show()

+----------+------------------+
|Product ID|       Total Sales|
+----------+------------------+
|    P00350|           65583.9|
|    P00225|53441.270000000004|
|    P00842|53168.240000000005|
|    P00415| 51834.43000000001|
|    P00743|51455.810000000005|
|    P00691|          50373.47|
|    P00186|50283.850000000006|
|    P00386| 49821.72999999999|
|    P00958|49193.369999999995|
|    P00340| 48476.54999999999|
+----------+------------------+



### Q16. Which customer segment (Regular, VIP) contributes the most to total sales?

In [23]:
# Group by Customer Segment and calculate the total sales
total_sales_per_segment = df.groupBy("Customer Segment") \
                            .agg(f.sum("Total Sales").alias("Total Sales"))

# Order by Total Sales in descending order
total_sales_per_segment = total_sales_per_segment.orderBy(f.col("Total Sales").desc())

# Show the result
total_sales_per_segment.show()

+----------------+--------------------+
|Customer Segment|         Total Sales|
+----------------+--------------------+
|         Regular|1.1948929059999974E7|
|             VIP|1.1559202930000018E7|
+----------------+--------------------+



### Q17. Analyze the sales trend over time for different product categories.

In [24]:
# Extract the year and month from the Date column
df = df.withColumn("Year", f.year(f.col("Date")))
df = df.withColumn("Month", f.month(f.col("Date")))

# Group by Category, Year, and Month, and calculate the total sales
sales_trend = df.groupBy("Category", "Year", "Month") \
                .agg(f.sum("Total Sales").alias("Total Sales"))

# Order the result by Category, Year, and Month
sales_trend = sales_trend.orderBy("Category", "Year", "Month")

sales_trend.show(5)

+--------+----+-----+------------------+
|Category|Year|Month|       Total Sales|
+--------+----+-----+------------------+
|Clothing|2022|    9|180573.57000000004|
|Clothing|2022|   10|         281169.85|
|Clothing|2022|   11|184325.97999999998|
|Clothing|2022|   12|188024.62999999998|
|Clothing|2023|    1|200487.02000000008|
+--------+----+-----+------------------+
only showing top 5 rows



### Q18. Which stores have the highest sales growth rate over the past year?

In [25]:
# Extract the year from the Date column
df = df.withColumn("Year", f.year(f.col("Date")))

# Calculate total sales for each store by year
sales_per_store_year = df.groupBy("Store ID", "Year") \
                         .agg(f.sum("Total Sales").alias("Total Sales"))

# Create a window specification to calculate the sales for the previous year
window_spec = Window.partitionBy("Store ID").orderBy("Year")

# Calculate the previous year's sales for each store
sales_per_store_year = sales_per_store_year.withColumn("Previous Year Sales", f.lag("Total Sales").over(window_spec))

# Calculate the growth rate as (current year sales - previous year sales) / previous year sales
sales_per_store_year = sales_per_store_year.withColumn("Growth Rate", 
                                                       (f.col("Total Sales") - f.col("Previous Year Sales")) / f.col("Previous Year Sales"))

# Filter out rows where growth rate cannot be calculated (e.g., the first year)
sales_per_store_year = sales_per_store_year.filter(f.col("Previous Year Sales").isNotNull())

# Identify the stores with the highest growth rate
top_growth_stores = sales_per_store_year.orderBy(f.col("Growth Rate").desc()).limit(10)

# Show the result
top_growth_stores.select("Store ID", "Year", "Total Sales", "Previous Year Sales", "Growth Rate").show()

+--------+----+------------------+-------------------+------------------+
|Store ID|Year|       Total Sales|Previous Year Sales|       Growth Rate|
+--------+----+------------------+-------------------+------------------+
|    S097|2023|144121.25999999998|            9851.29|13.629684031228393|
|    S053|2023|130308.31000000001|           18299.13|  6.12101121747318|
|    S061|2023|127207.56999999996|           20900.41| 5.086367205236642|
|    S040|2023|125535.66999999998| 22710.819999999996|4.5275709991977395|
|    S096|2023|         137113.59|           24839.58| 4.519964105673284|
|    S003|2023|105611.00999999995|           20788.22| 4.080329628991802|
|    S077|2023|122598.29999999999| 25560.850000000006| 3.796331107924813|
|    S039|2023|167740.20999999993|           36790.56| 3.559327447040761|
|    S041|2023|          141922.1| 32321.710000000006|3.3909217674436154|
|    S055|2023|113229.07999999999|           27062.88| 3.183925731481645|
+--------+----+------------------+----

### Q19. What is the average purchase amount by different age groups?

In [26]:
# Define age groups using a case-when statement
df = df.withColumn("Age Group", f.when(f.col("Customer Age").between(18, 25), "18-25")
                                 .when(f.col("Customer Age").between(26, 35), "26-35")
                                 .when(f.col("Customer Age").between(36, 45), "36-45")
                                 .when(f.col("Customer Age").between(46, 55), "46-55")
                                 .when(f.col("Customer Age").between(56, 65), "56-65")
                                 .otherwise("66+"))

# Group by Age Group and calculate the average purchase amount
avg_purchase_by_age_group = df.groupBy("Age Group") \
                              .agg(f.avg("Total Sales").alias("Average Purchase Amount"))

# Order the results by Age Group (optional)
avg_purchase_by_age_group = avg_purchase_by_age_group.orderBy("Age Group")

# Show the result
avg_purchase_by_age_group.show()

+---------+-----------------------+
|Age Group|Average Purchase Amount|
+---------+-----------------------+
|    18-25|        2382.6800729927|
|    26-35|      2335.254928533614|
|    36-45|     2332.2464622897455|
|    46-55|      2397.124820592824|
|    56-65|      2341.437943974627|
|      66+|      2291.897156448201|
+---------+-----------------------+



### Q20. Which product sub-categories have the highest return rate (e.g., returns not explicitly stated but implied by negative sales or quantity adjustments)?

In [27]:
# Calculate Total w/ Discount and Returns
df = df.withColumn("Total w/ Discount", f.col("Quantity Sold") * f.col("Price"))
df = df.withColumn("Returns", (f.col("Total w/ Discount") - f.col("Total Sales")))
df = df.withColumn("Rate", (f.col("Returns") / f.col("Total w/ Discount")))

# Group by Sub-Category and aggregate the results
returns_per_subcategory = df.groupBy("Sub-Category") \
                            .agg(f.sum("Total w/ Discount").alias("Total w/ Discount"), 
                                 f.sum("Returns").alias("Total Returns"),
                                 f.sum("Rate").alias("Total Rate"))

# Order by Total Returns in descending order and show the highest result
highest_returns_subcategory = returns_per_subcategory.orderBy(f.col("Total Rate").desc())

# Show the result
highest_returns_subcategory.show()

+--------------+------------------+------------------+-----------------+
|  Sub-Category| Total w/ Discount|     Total Returns|       Total Rate|
+--------------+------------------+------------------+-----------------+
|       Jackets|1486856.7999999998|231598.94000000006|82.75969566421996|
|Action Figures|1535918.0100000019|230601.69000000024| 81.6704855955365|
|       Tablets|1439143.2700000003| 217950.5900000001|79.34026674627493|
|        Chairs|1340902.2499999995|211954.96000000002|77.97003638421117|
|         Dairy|1438172.6700000009|213714.47999999992|77.70003154228694|
|         Dolls|1376544.3200000008|212751.11999999982|76.95002562054958|
|         Sofas|1475339.1499999987|         217679.78|76.87001793494086|
|       Cameras|1457548.6799999997|214623.08999999994|76.26015666068736|
|       Laptops|1491591.0799999994|225388.23000000007|76.19047798904171|
|   Smartphones| 1344566.949999999|197502.31999999998|75.90975895747852|
|         Shoes| 1345162.350000001|197592.409999999

### Q21. What is the relationship between discount percentage and customer feedback rating?

In [28]:
df = df.withColumn("Discount Percentage", f.col("Price")*f.col("Discount") / f.col("Price"))

feedback_rating_disctount = df.groupBy("Feedback Rating")\
                               .agg(f.avg("Discount Percentage").alias("Avarage Discount Rage"))\
                               .orderBy(f.col("Feedback Rating").desc())

feedback_rating_disctount.show()

+---------------+---------------------+
|Feedback Rating|Avarage Discount Rage|
+---------------+---------------------+
|              5|  0.15358868894601532|
|              4|  0.15091046277665982|
|              3|  0.14770833333333283|
|              2|  0.14994017946161448|
|              1|   0.1492469437652808|
+---------------+---------------------+



### Q22. How does the customer loyalty program impact total sales and frequency of purchases?

In [29]:
# Group by Loyalty Program and calculate total sales and frequency of purchases
loyalty_program_analysis = df.groupBy("Loyalty Program") \
                             .agg(f.sum("Total Sales").alias("Total Sales"),
                                  f.count("Transaction ID").alias("Frequency of Purchases"))

# Show the result
loyalty_program_analysis.show()

+---------------+--------------------+----------------------+
|Loyalty Program|         Total Sales|Frequency of Purchases|
+---------------+--------------------+----------------------+
|             No|1.1867346010000013E7|                  5038|
|            Yes|1.1640785979999976E7|                  4962|
+---------------+--------------------+----------------------+



### Q23. Which cities have the highest number of VIP customers?

In [30]:
# Extract the city from the "Store Location" column
df = df.withColumn("City", f.split(f.col("Store Location"), ", ").getItem(0))

# Filter for VIP customers and group by City
vip_customers_cities = df.filter(f.col("Customer Segment") == "VIP")\
                         .groupBy("City")\
                         .agg(f.count("*").alias("VIP_Count"))\
                         .orderBy(f.col("VIP_Count").desc())

# Show the result
vip_customers_cities.show()

+------------------+---------+
|              City|VIP_Count|
+------------------+---------+
|     New Carolfurt|       68|
|  South Cherylside|       67|
|  Port Danielburgh|       63|
|         Nancyport|       62|
|         Huangside|       61|
|      Wheelermouth|       60|
|       Taylorhaven|       59|
|        Port Jerry|       59|
|        Lake Diana|       58|
|North Karenborough|       58|
|  North Stevenland|       58|
|        North John|       58|
|          Seantown|       58|
|        West Ellen|       57|
|      Jonesborough|       57|
|       East Amanda|       57|
|   South Sandraton|       57|
|        Duncanstad|       56|
|        Booneshire|       56|
|    West Haleystad|       56|
+------------------+---------+
only showing top 20 rows



### Q24. Identify any seasonal trends in sales for specific product categories.

In [31]:
# Extract the year and month from the Date column
df = df.withColumn("Year", f.year(f.col("Date")))
df = df.withColumn("Month", f.month(f.col("Date")))

seasonal_sales = df.groupBy("Category", "Year", "Month") \
                            .agg(f.sum("Total Sales").alias("Total Sales")) \
                            .orderBy(f.col("Month").desc())

seasonal_sales.show()

+-----------+----+-----+------------------+
|   Category|Year|Month|       Total Sales|
+-----------+----+-----+------------------+
|       Toys|2022|   12| 193910.4599999999|
|Electronics|2023|   12|181577.18000000002|
|       Toys|2023|   12|142413.12999999995|
|  Furniture|2022|   12|         274499.61|
|Electronics|2022|   12|210889.59000000008|
|   Clothing|2023|   12|         208572.44|
|  Groceries|2022|   12| 174498.4499999999|
|   Clothing|2022|   12|188024.62999999998|
|  Furniture|2023|   12|148003.99999999997|
|  Groceries|2023|   12|190960.07999999987|
|   Clothing|2023|   11|214286.96999999988|
|       Toys|2022|   11|149200.50000000003|
|  Furniture|2022|   11|          185598.3|
|  Groceries|2022|   11|194937.44000000003|
|       Toys|2023|   11|208331.46999999997|
|Electronics|2022|   11|254889.34000000005|
|   Clothing|2022|   11|184325.97999999998|
|  Furniture|2023|   11|169109.98000000004|
|Electronics|2023|   11|181657.73000000007|
|  Groceries|2023|   11|        

### Q25. What is the average transaction amount for online stores compared to physical stores?

In [32]:
# Create a new column "Store Category" based on "Store Type"
df = df.withColumn("Store Category", 
                   f.when(f.col("Store Type") == "Online", "Online")
                    .when(f.col("Store Type").isin("Outlet", "Supermarket"), "Physical")
                    .otherwise("Other"))

# Filter to include only "Online" and "Physical"
filtered_df = df.filter(f.col("Store Category").isin("Online", "Physical"))

# Group by "Store Category" and calculate the average transaction amount
avg_transaction_amount = filtered_df.groupBy("Store Category")\
                                   .agg(f.avg("Total Sales").alias("Average Total Sales"))

# Show the result
avg_transaction_amount.show()

+--------------+-------------------+
|Store Category|Average Total Sales|
+--------------+-------------------+
|        Online|  2375.496617210682|
|      Physical| 2338.2667254901958|
+--------------+-------------------+



### Q26. Analyze the relationship between store location and the preferred payment method.

In [33]:
# Get the Country
df = df.withColumn("Country", f.split(f.col("Store Location"), ", ").getItem(1))

# Calculate the frequency of Payment Method per Country
location_payment_method = df.groupBy("Country")\
                            .pivot("Payment Method") \
                            .agg(f.count("Payment Method").alias("Transaction Count"))

# Order by Country
location_payment_method = location_payment_method.orderBy("Country")

# Show the result
location_payment_method.show()

+--------------------+----+-----------+--------------+
|             Country|Cash|Credit Card|Online Payment|
+--------------------+----+-----------+--------------+
|         Afghanistan|  33|         36|            39|
|             Albania|  70|         80|            65|
|             Andorra|  42|         38|            32|
|              Angola|  30|         27|            39|
|            Anguilla|  31|         35|            31|
|Antarctica (the t...|  35|         24|            29|
| Antigua and Barbuda|  19|         28|            35|
|             Austria|  37|         50|            28|
|             Bahamas|  37|         38|            22|
|             Bahrain|  34|         37|            43|
|             Belarus|  25|         34|            32|
|               Benin|  30|         23|            44|
|             Bermuda|  39|         30|            37|
|              Bhutan|  70|         64|            81|
|British Virgin Is...|  32|         39|            28|
|         

### Q27. What is the average number of items purchased per transaction across different store types?

In [34]:
number_items_store_types = df.groupBy("Store Type").agg(f.sum("Quantity Sold").alias("Total Sold"))

number_items_store_types.show()

+-----------+----------+
| Store Type|Total Sold|
+-----------+----------+
|Supermarket|     18283|
|     Online|     18635|
|     Outlet|     18167|
+-----------+----------+



### Q28. Identify the top 5 employees with the highest total sales and analyze their performance trends.

In [35]:
df = df.withColumn("Discount Percentage", f.col("Price")*f.col("Discount") / f.col("Price"))

top_5_employees = df.groupBy("Employee ID", "Store Type")\
                    .agg(f.sum("Total Sales").alias("Total Sales"),
                         f.sum("Quantity Sold").alias("Total Quantity Sold"),
                         f.avg("Price").alias("Avarage Price"),
                         f.avg("Discount Percentage").alias("Avarage Discount Rage"),
                         f.avg("Feedback Rating").alias("Avarage Feedback Rating"),
                        ).orderBy(f.col("Total Sales").desc())

top_5_employees.show(5)

+-----------+-----------+------------------+-------------------+-----------------+---------------------+-----------------------+
|Employee ID| Store Type|       Total Sales|Total Quantity Sold|    Avarage Price|Avarage Discount Rage|Avarage Feedback Rating|
+-----------+-----------+------------------+-------------------+-----------------+---------------------+-----------------------+
|      E0142|     Outlet|          49037.76|                 76|685.9381818181818|  0.13272727272727275|     3.3636363636363638|
|      E0020|     Online|          45213.45|                 94|       519.484375|             0.114375|                  3.125|
|      E0363|     Outlet|45125.729999999996|                 77|633.3272727272728|   0.1372727272727273|     1.9090909090909092|
|      E0079|Supermarket|          43857.33|                 75|654.2819999999999|                0.129|                    2.7|
|      E0486|     Online|43830.729999999996|                 89|628.4393333333334|  0.10533333333

### Q29. What is the correlation between customer age and the likelihood of purchasing high-value products?

In [36]:
# Define age groups using a case-when statement
df = df.withColumn("Age Group", f.when(f.col("Customer Age").between(18, 25), "18-25")
                                 .when(f.col("Customer Age").between(26, 35), "26-35")
                                 .when(f.col("Customer Age").between(36, 45), "36-45")
                                 .when(f.col("Customer Age").between(46, 55), "46-55")
                                 .when(f.col("Customer Age").between(56, 65), "56-65")
                                 .otherwise("66+"))

# Group by Age Group and calculate the average purchase amount
avg_purchase_by_age_group = df.groupBy("Age Group") \
                              .agg(f.avg("Total Sales").alias("Average Purchase Amount"))

# Order the results by Age Group (optional)
avg_purchase_by_age_group = avg_purchase_by_age_group.orderBy("Age Group")

# Calculate the average purchase amount by age group (already done)
avg_purchase_by_age_group = df.groupBy("Age Group") \
                              .agg(f.avg("Total Sales").alias("Average Purchase Amount"))

# Join the original DataFrame with the average purchase by age group to get the "Average Purchase Amount" column
df_with_avg = df.join(avg_purchase_by_age_group, on="Age Group", how="inner")

# Filter out customers who bought above the average in their respective age groups
df_above_avg = df_with_avg.filter(f.col("Total Sales") > f.col("Average Purchase Amount"))

# Count the total number of customers in each age group who bought above the average
count_above_avg = df_above_avg.groupBy("Age Group") \
                              .agg(f.count("Customer ID").alias("Total Above Average"))

# Calculate the total number of customers in each age group
total_in_age_group = df.groupBy("Age Group") \
                       .agg(f.count("Customer ID").alias("Total Customers"))

# Calculate the percentage of customers in each age group who bought above the average
percentage_above_avg = count_above_avg.join(total_in_age_group, on="Age Group", how="inner") \
                                      .withColumn("Percentage Above Average", 
                                                  (f.col("Total Above Average") / f.col("Total Customers")) * 100)

# Combine all the information into a final DataFrame
final_df = avg_purchase_by_age_group.join(percentage_above_avg, on="Age Group", how="inner") \
                                    .select("Age Group", 
                                            "Average Purchase Amount", 
                                            "Total Above Average", 
                                            "Percentage Above Average")

# Step 8: Order the results by Age Group (optional)
final_df = final_df.orderBy("Age Group")

# Step 9: Show the result
final_df.show()

+---------+-----------------------+-------------------+------------------------+
|Age Group|Average Purchase Amount|Total Above Average|Percentage Above Average|
+---------+-----------------------+-------------------+------------------------+
|    18-25|        2382.6800729927|                613|       40.67684140676842|
|    26-35|      2335.254928533614|                765|      40.497617787188986|
|    36-45|     2332.2464622897455|                744|       40.36896364622897|
|    46-55|      2397.124820592824|                764|       39.72958918356734|
|    56-65|      2341.437943974627|                757|       40.01057082452431|
|      66+|      2291.897156448201|                376|       39.74630021141649|
+---------+-----------------------+-------------------+------------------------+



### Q30. What is the average feedback rating for each product category, and how does it correlate with the average discount given?

In [37]:
# Calculate average feedback rating and average discount by product category
category_feedback_discount = df.groupBy("Category") \
    .agg(f.avg("Feedback Rating").alias("Average Feedback Rating"),
         f.avg("Discount").alias("Average Discount"))

# Show the result
category_feedback_discount.show()

# Optionally, calculate correlation between Average Feedback Rating and Average Discount
correlation = category_feedback_discount.stat.corr("Average Feedback Rating", "Average Discount")
print(f"The correlation between average feedback rating and average discount is: {correlation}")

+-----------+-----------------------+-------------------+
|   Category|Average Feedback Rating|   Average Discount|
+-----------+-----------------------+-------------------+
|  Groceries|      2.983143282102132|0.14816063460585016|
|Electronics|     2.9493857493857494|0.15120393120393072|
|   Clothing|     2.9721115537848606|0.15178286852589584|
|  Furniture|      2.998481781376518|0.14925607287449394|
|       Toys|      2.988798370672098|0.15084521384928679|
+-----------+-----------------------+-------------------+

The correlation between average feedback rating and average discount is: -0.5414582277778561
